In [3]:
!pip install azureml-sdk

  Using cached azureml_sdk-1.59.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached azureml_core-1.59.0.post2-py3-none-any.whl.metadata (3.4 kB)
  Using cached azureml_dataset_runtime-1.59.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached azureml_train_core-1.59.0-py3-none-any.whl.metadata (1.8 kB)
INFO: pip is looking at multiple versions of azureml-sdk to determine which version is compatible with other requirements. This could take a while.
  Using cached azureml_sdk-1.58.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached azureml_core-1.58.0.post1-py3-none-any.whl.metadata (3.2 kB)
  Using cached azureml_dataset_runtime-1.58.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached azureml_train_core-1.58.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached azureml_sdk-1.57.0-py3-none-any.whl.metadata (3.5 kB)
  Using cached azureml_core-1.57.0.post3-py3-none-any.whl.metadata (3.1 kB)
  Using cached azureml_dataset_runtime-1.57.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached azureml

In [5]:
!pip install azureml-core

  Using cached azureml_core-1.59.0.post2-py3-none-any.whl.metadata (3.4 kB)
  Using cached backports.tempfile-1.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached pathspec-0.12.1-py3-none-any.whl.metadata (21 kB)
  Using cached azure_core-1.32.0-py3-none-any.whl.metadata (39 kB)
  Using cached paramiko-3.5.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached azure_mgmt_resource-23.3.0-py3-none-any.whl.metadata (41 kB)
  Using cached azure_mgmt_authorization-4.0.0-py3-none-any.whl.metadata (18 kB)
  Using cached azure_graphrbac-0.61.2-py2.py3-none-any.whl.metadata (11 kB)
  Using cached azure_common-1.1.28-py2.py3-none-any.whl.metadata (5.0 kB)
  Using cached msrest-0.7.1-py3-none-any.whl.metadata (21 kB)
  Using cached msrestazure-0.6.4.post1-py2.py3-none-any.whl.metadata (15 kB)
  Using cached ndg_httpsclient-0.5.1-py3-none-any.whl.metadata (6.2 kB)
  Using cached SecretStorage-3.3.3-py3-none-any.whl.metadata (4.0 kB)
  Using cached jsonpickle-4.0.5-py3-none-any.whl.metadata (8.2 

In [227]:
from azureml.core import Workspace
import json

#id = open('key.json', 'r')
#mi = json.load(id)

ws = Workspace.create(name="bankrupt_dataprophet_v2",
                      subscription_id = "f7573334-1f4f-4ce7-9b75-ce1f32e6bd37",
                      resource_group = "group_1",
                      location ="centralindia")

WorkspaceException: WorkspaceException:
	Message: Workspace with name 'bankrupt_dataprophet_v2' already exists under resource group with name 'group_1'.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Workspace with name 'bankrupt_dataprophet_v2' already exists under resource group with name 'group_1'."
    }
}

In [249]:
from azureml.core import Workspace

ws = Workspace.get(
    name="bankrupt_dataprophet_v2",
    subscription_id="f7573334-1f4f-4ce7-9b75-ce1f32e6bd37",
    resource_group="group_1"
)


In [250]:
from azureml.core.model import Model

registered_model = Model.register(model_path="model.pkl",
                                  model_name="bankrupt_predict",
                                  workspace=ws)

Registering model bankrupt_predict


In [251]:
model = Model(workspace=ws, name="bankrupt_predict")
print(model.version)  # Confirm it's the latest one


50


In [252]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

env = Environment("bankrupt_env_v2")
deps = CondaDependencies.create(
    conda_packages=["python=3.8", "numpy", "pandas", "scikit-learn", "joblib", "lightgbm"],
    pip_packages=["azureml-defaults"]
)
env.python.conda_dependencies = deps


In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

inference_config = InferenceConfig(entry_script="score.py", environment=env)
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name="bankrupt-service-v2",
                       models=[model],  # 👈 Pass the model object
                       inference_config=inference_config,
                       deployment_config=deployment_config)


C:\Users\david\AppData\Local\Temp\ipykernel_21460\295921370.py:7: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


WebserviceException: WebserviceException:
	Message: Service bankrupt-service-v2 with the same name already exists, please use a different service name or delete the existing service.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service bankrupt-service-v2 with the same name already exists, please use a different service name or delete the existing service."
    }
}

: 

In [ ]:
service.wait_for_deployment()

Performing interactive authentication. Please follow the instructions on the terminal.


AuthenticationException: AuthenticationException:
	Message: Please ensure you have network connection. Error detail: HTTPSConnectionPool(host='login.microsoftonline.com', port=443): Max retries exceeded with url: /organizations/v2.0/.well-known/openid-configuration (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000015BBEEB9850>: Failed to resolve 'login.microsoftonline.com' ([Errno 11001] getaddrinfo failed)"))
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "inner_error": {
            "code": "Authentication"
        },
        "message": "Please ensure you have network connection. Error detail: HTTPSConnectionPool(host='login.microsoftonline.com', port=443): Max retries exceeded with url: /organizations/v2.0/.well-known/openid-configuration (Caused by NameResolutionError(\"<urllib3.connection.HTTPSConnection object at 0x0000015BBEEB9850>: Failed to resolve 'login.microsoftonline.com' ([Errno 11001] getaddri

In [ ]:
from azureml.core.webservice import Webservice

service = Webservice(workspace=ws, name="bankrupt-service")
print(service.get_logs())


None


In [ ]:
scoring_uri = service.scoring_uri
print("Scoring URI:", scoring_uri)

Scoring URI: http://3603dd1c-1fa9-4e6e-a2f6-6d0194d40237.centralindia.azurecontainer.io/score


In [ ]:
print(service.get_logs())


/bin/bash: /azureml-envs/azureml_41f9d9d58f0c63eedf01ae123fa38f8e/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_41f9d9d58f0c63eedf01ae123fa38f8e/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_41f9d9d58f0c63eedf01ae123fa38f8e/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2025-04-11T18:33:04,504901387+00:00 - rsyslog/run 
bash: /azureml-envs/azureml_41f9d9d58f0c63eedf01ae123fa38f8e/lib/libtinfo.so.6: no version information available (required by bash)
2025-04-11T18:33:04,512004460+00:00 - gunicorn/run 
2025-04-11T18:33:04,513986603+00:00 | gunicorn/run | 
2025-04-11T18:33:04,515846848+00:00 - nginx/run 
2025-04-11T18:33:04,517749032+00:00 | gunicorn/run | ###############################################
2025-04-11T18:33:04,519231894+00:00 | gunicorn/run | AzureML Container Runtime Information
2025-04-11T18:33:04,521252518+00:00 | gunico